In [171]:
from IPython.display import HTML
from sklearn.metrics import accuracy_score
import pandas as pd
from sklearn.model_selection import cross_val_score,KFold, cross_val_predict,cross_validate
import xgboost as xgb
import warnings

proba_0 = []
proba_1 = []

warnings.filterwarnings("ignore", category=UserWarning)

df_train = pd.read_csv("train.csv")
df_train.head(10)
df_test = pd.read_csv("test.csv")
x_train =df_train[df_train.columns[1:11]]
x_test = df_test[df_test.columns[1:11]]
y_test = df_test['steps_title']
y_train = df_train['steps_title']
kfold = KFold(n_splits=10)
features_train = df_train.columns[1:11]
clf = xgb.XGBClassifier(seed=42, subsample=0.9)
cv_results = cross_val_score(clf,x_train , y_train,cv=kfold , scoring='accuracy')
output = cross_validate(clf, x_train, y_train, cv=10, scoring = 'accuracy', return_estimator =True)
y_pred = cross_val_predict(clf,x_test,y_test,cv=10)
proba = cross_val_predict(clf,x_test,y_test,cv=10, method='predict_proba')
clf.fit(x_train , y_train , verbose=0, eval_set =[(x_test, y_test)])
print("accuracy : ", accuracy_score(y_test, y_pred))
print(pd.crosstab(y_test,y_pred))
df_test["prediction"] = y_pred

for prob0 , prob1 in proba:
    proba_0.append(float(prob0))
    proba_1.append(float(prob1))
df_test['Prob_0'] = proba_0
df_test['Prob_1'] = proba_1
final_df = (df_test.sort_values(by=['Prob_0'], ascending=False)).iloc[:,[0 , 13]]
HTML(final_df.to_html(index=False))

accuracy :  0.7712765957446809
col_0         2   3
steps_title        
2            71  23
3            20  74


job_applicant_id,Prob_0
46078,0.998020
3432,0.997730
46021,0.997584
46119,0.996399
46097,0.995586
47211,0.995352
32188,0.994726
43721,0.993912
46185,0.993615
43185,0.993067
